[Python 最適化](https://techacademy.jp/magazine/20619)

In [1]:
!pip install pulp

     |████████████████████████████████| 41.2 MB 76 kB/s 


## 例題
あるレストランで、手持ちの材料からハンバーグとオムレツを作って利益を最大にしたいと考えています。手持ちの材料は以下の通りです。

* ひき肉 3800 [g]
* タマネギ 2100 [g]
* ケチャップ 1200 [g]  

商品を作るのに必要な材料の量は以下の通りです。

* ハンバーグ 1 個あたり、ひき肉 60 [g]、タマネギ 20 [g]、ケチャップ 20 [g]
* オムレツ 1 個あたり、ひき肉 40 [g]、タマネギ 30 [g]、ケチャップ 10 [g]  

販売価格は以下の通りです。

* ハンバーグ 400 [円/個]
* オムレツ 300 [円/個]

総売上を最大にするには、ハンバーグとオムレツを幾つずつ作れば良いでしょうか。

In [2]:
from pulp.pulp import LpProblem
from pulp.pulp import LpVariable
from pulp.constants import LpMaximize

In [ ]:
# 今回の問題では総和を最大化するので LpMaximize を指定する
problem = LpProblem('Restaurant', LpMaximize)


In [ ]:
# 使用する変数
hamburg = LpVariable('Hamburg')
omelet = LpVariable('Omelet')

In [ ]:
# 目的関数
problem += 400 * hamburg + 300 * omelet

# 制約条件
problem += 60 * hamburg + 40 * omelet <= 3800  # ひき肉の総量は 3800g
problem += 20 * hamburg + 30 * omelet <= 2100  # 玉ねぎの総量は 2100g
problem += 20 * hamburg + 10 * omelet <= 1200  # ケチャップの総量は 1200g

In [ ]:
# 解く
problem.solve()

# 結果表示
print('ハンバーグの個数: {hamburg}'.format(hamburg=hamburg.value()))
print('オムレツの個数: {omelet}'.format(omelet=omelet.value()))

ハンバーグの個数: 30.0
オムレツの個数: 50.0




---



---



# 例題2（線形最適化問題）
https://mikiokubo.github.io/analytics/optimization.html

* トンコケ，コケトン，ミックスの丼を販売
* 資源制約の下で，利益を最大化

変数

* トンコケ丼 $x_1$ ，コケトン丼 $x_2$​ ，ミックス丼 $x_3$
 
* 定式化  
$maximize$ $15x_1+18x_2+30x_3$  
$s.t.$  
$2x_1+x_2+x_3<=60$  
$x_1+2x_2+x_3<=60$  
$x_3<=30$  
$x_1,x_2,x_3>=0$


In [3]:
# 今回の問題では総和を最大化するので LpMaximize を指定する
problem = LpProblem('Restaurant', LpMaximize)


In [4]:
# 使用する変数
x1 = LpVariable('x1')
x2 = LpVariable('x2')
x3 = LpVariable('x3')

In [5]:
# 目的関数
problem += 15 * x1 + 18 * x2 + 30 * x3

# 制約条件
problem += 2 * x1 + 1 * x2 + 1 * x3 <= 60
problem += 1 * x1 + 2 * x2 + 1 * x3 <= 60
problem += x3 <= 30
problem += x1 >= 0
problem += x2 >= 0
problem += x3 >= 0

In [6]:
# 解く
problem.solve()

# 結果表示
print(f'x1: {x1.value()}')
print(f'x2: {x2.value()}')
print(f'x3: {x3.value()}')

x1: 10.0
x2: 10.0
x3: 30.0


別の解き方

In [9]:
!pip install gurobipy

     |████████████████████████████████| 11.1 MB 4.9 MB/s 


In [10]:
from gurobipy import GRB, Model, quicksum, multidict, tuplelist

In [11]:
model = Model('lo1')

x1 = model.addVar(name='x1')
x2 = model.addVar(name='x2')
x3 = model.addVar(ub=30., name='x3')

model.update() #Gurobiの怠惰な更新(lazy update)という仕様（忘れずに！）

Restricted license - for non-production use only - expires 2022-01-13


In [12]:
model.addConstr(2*x1 + x2 + x3 <= 60)
# 別の定義方法 1
#L1 = LinExpr([2,1,1],[x1,x2,x3]) #線形表現(式）
# 別の定義方法 2
#L1 = LinExpr()     #線形表現(式）
#L1.addTerms(2,x1)  #項を追加
#L1.addTerms(1,x2)
#L1.addTerms(1,x3)
#model.addConstr(lhs=L1,sense='<',rhs=60)  #制約を追加

model.addConstr(x1 + 2*x2 + x3 <= 60)

model.setObjective(15*x1 + 18*x2 + 30*x3, GRB.MAXIMIZE)

#SCIPを使用したい場合
# import pulp
# solver = pulp.SCIP()
# model.optimize(solver)

model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 2 rows, 3 columns and 6 nonzeros
Model fingerprint: 0x1da73bc0
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+01, 3e+01]
  Bounds range     [3e+01, 3e+01]
  RHS range        [6e+01, 6e+01]
Presolve time: 0.02s
Presolved: 2 rows, 3 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.3000000e+31   3.000000e+30   3.300000e+01      0s
       3    1.2300000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds
Optimal objective  1.230000000e+03


In [13]:
if model.Status == GRB.Status.OPTIMAL:
    print('Opt. Value=',model.ObjVal)
    for v in model.getVars():
        print(v.VarName,v.X)

Opt. Value= 1230.0
x1 10.0
x2 10.0
x3 30.0
